# Revise UC formulation with AMS

In [1]:
import andes
import ams

In [2]:
andes.__version__

'1.9.2'

In [3]:
ams.__version__

'0.9.10'

In [4]:
ams.config_logger(stream_level=20)

In [5]:
sp = ams.load('../cases/ieee39_uced_vsg.xlsx',
              setup=True,
              no_output=True,
              default_config=True)

Parsing input file "../cases/ieee39_uced_vsg.xlsx"...
Input file parsed in 0.0727 seconds.
System set up in 0.0012 seconds.


## System inspection

There are four IBRs `REGCV1`

In [6]:
sp.REGCV1.as_df()

,idx,u,name,bus,gen,Sn,gammap,gammaq,M,D,Mmax,Dmax
uid,,,,,,,,,,,,
0,VSG_1,1.0,VSG_1,30,PV_30,1040.0,1.0,1.0,87.36000,5.2000,104.000,10.400
1,VSG_2,1.0,VSG_2,35,PV_35,1085.7,1.0,1.0,75.56472,5.4285,108.570,10.857
2,VSG_3,1.0,VSG_3,37,PV_37,970.2,1.0,1.0,47.15172,4.8510,97.020,9.702
3,VSG_4,1.0,VSG_4,38,PV_38,1684.1,1.0,1.0,116.20290,8.4205,252.615,16.841


There are ten units and 24 time periods.

The time periods data are stored in the model `UCTSlot`

In [7]:
sp.UCTSlot.as_df()

,idx,u,name,sd
uid,,,,
0,UCT_1,1.0,UCT 1,"0.641,0.0"
1,UCT_2,1.0,UCT 2,"0.634,0.0"
2,UCT_3,1.0,UCT 3,"0.623,0.0"
3,UCT_4,1.0,UCT 4,"0.615,0.0"
4,UCT_5,1.0,UCT 5,"0.62,0.0"
5,UCT_6,1.0,UCT 6,"0.62,0.0"
6,UCT_7,1.0,UCT 7,"0.665,0.0"
7,UCT_8,1.0,UCT 8,"0.713,0.0"
8,UCT_9,1.0,UCT 9,"0.728,0.0"


## Revise formulation 

Add decision variables

Take the M as an example, following explanations:
- `name=M` means the variable is named `M`, and later you should be able to access it thourgh `UC.M`
- `model=VSG` means the variable is associated with the group `VSG` (a model name is also acceptable)
- `src=M` means the variable values will be set to `VSG.M` after successful solving
- `horizon=sp.UCTSlot` means the variable indexes `sp.UCTSlot` as its time dimension
- `nonneg=True` means the variable is non-negative

In [8]:
sp.UC.addVars(name='M',
              model='VSG', src='M', 
              horizon=sp.UCTSlot,
              nonneg=True)

Var: VSG.M

Add constraints

Take the Mlb as an example, following explanations:
- `name=Mlb` means the constraint is named `Mlb`, and later you should be able to access it thourgh `UC.Mlb`
- `info` is a brief description of the constraint
- `e_str` is the descriptive string of the constraint,
- `is_eq=False` means the constraint is an inequality constraint

In the last, the mathematical expression of the constraint is `-sum(M) + 0.1 <= 0`

In [9]:
sp.UC.addConstrs(name='Mlb', info='Inertia lower bound',
                 e_str='-sum(M, axis=0) + 0.1', is_eq=False)

Constraint: Mlb [ON]

## Solve the revised UC

Initialize the revised UC

In [10]:
sp.UC.init()

All generators are online at initial, make initial guess for commitment.
Turn off StaticGen ['PV_37' 'PV_34'] as initial commitment guess.
<UC> initialized in 0.0179 seconds.


True

In [11]:
sp.UC.run(solver='GUROBI')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25


<UC> solved as optimal in 0.1018 seconds, converged in 11 iterations with GUROBI.


True

After sovling it, we can inspect the UC results.

For example, following is the virtual inertia of first time period's results:

In [12]:
sp.UC.get(src='M', attr='v', idx=sp.REGCV1.idx.v, horizon=['UCT_1'])

array([1., 0., 0., 0.])